In [1]:
import pandas as pd

In [2]:
df_fraud_detection = pd.read_pickle("data/cleaned_transaction.pk1")
df_fraud_detection = df_fraud_detection.copy()

In [3]:
#df_fraud_detection[
    #(df_fraud_detection["sender_country"] == "Sweden") &
    #(df_fraud_detection["transaction_type"] == "outgoing")]

In [4]:
# High amount detection
high_amount_threshold = df_fraud_detection["amount"].quantile(0.98)
# high_amount_threshold = 199999

with open("logging_folder/high_amount_log", "a") as log:
    for row in df_fraud_detection.itertuples():
        if (
            row.sender_country == "Sweden" and
            row.transaction_type == "outgoing" and
            row.amount > high_amount_threshold
        ):
            log.write(f"{row.transaction_id}: Amount flagged {row.amount}\n")
    print("Loggen för höga belopp har fyllts i.")

Loggen för höga belopp har fyllts i.


In [5]:
# Country based warnings
black_list = ["Iran", "Myanmar", "Democratic People's Republic of Korea"]

gray_list = [
    "Algeria", "Angola", "Bulgaria", "Burkina Faso", "Cameroon",
    "Côte d'Ivoire", "Croatia", "Democratic Republic of Congo", "Haiti", "Kenya",
    "Lao People's Democratic Republic", "Lebanon", "Mali", "Monaco", "Mozambique",
    "Namibia", "Nepal", "Nigeria", "South Africa", "South Sudan", "Syria",
    "Tanzania", "Venezuela", "Vietnam", "Yemen"
]

with open("logging_folder/blacklisted_countrys_log", "a") as blacklisted_log, \
     open("logging_folder/graylisted_countries_log", "a") as graylisted_log, \
     open("logging_folder/missing_country_log", "a") as missing_country_log:

    for row in df_fraud_detection.itertuples():
        receiver_country_str = str(row.receiver_country).strip()

        if row.sender_country == "Sweden" and row.transaction_type == "outgoing":
            if receiver_country_str in black_list:
                blacklisted_log.write(
                    f"{row.transaction_id}: Warning Blacklisted {row.receiver_country}: Amount sent {row.amount}\n"
                )
            elif receiver_country_str in gray_list:
                graylisted_log.write(
                    f"{row.transaction_id}: Warning Graylisted {row.receiver_country}: Amount sent {row.amount}\n"
                )
            elif receiver_country_str == "" or pd.isna(row.receiver_country):
                missing_country_log.write(
                    f"{row.transaction_id}: Warning Missing Country: Amount sent {row.amount}\n"
                )

In [6]:
# Frequent transactions detection
df_sweden_outgoing = df_fraud_detection[
    (df_fraud_detection["sender_country"] == "Sweden") &
    (df_fraud_detection["transaction_type"] == "outgoing")
].copy()

df_sweden_outgoing["date"] = df_sweden_outgoing["timestamp"].dt.date

daily_counts = df_sweden_outgoing.groupby(["sender_account", "date"]).size().reset_index(name="daily_transaction_count")

df_sweden_outgoing = df_sweden_outgoing.merge(daily_counts, on=["sender_account", "date"])

with open("logging_folder/frequent_amount_transactions_log", "a") as log:
    for row in df_sweden_outgoing.itertuples():
        if row.daily_transaction_count > 35:
            log.write(
                f"{row.transaction_id}: {row.sender_account} gjorde {row.daily_transaction_count} transaktioner den {row.date}\n"
            )

In [7]:
# Odd notes warning
suspicious_keywords = ["gift","test", "ok", "cash"]
amount_threshold = 8000
log_notes_amount_condition = "above"

with open("logging_folder/odd_notes_detection_log.txt", "a") as odd_notes_log:
    for row in df_sweden_outgoing.itertuples():
        current_notes = str(row.notes).strip()
        should_check_notes = False
        amount_condition_description = ""

        if log_notes_amount_condition == "above":
            if row.amount > amount_threshold:
                should_check_notes = True
                amount_condition_description = f"Amount higher than {amount_threshold}"
        elif log_notes_amount_condition == "below":
            if row.amount < amount_threshold:
                should_check_notes = True
                amount_condition_description = f"Amount < {amount_threshold:.2f}"
        else:
            print(f"Warning: Above threshold '{log_notes_amount_condition}'")
            break

        if should_check_notes:
            log_prefix = f"{row.transaction_id} {amount_condition_description} "

            if current_notes == "":
                odd_notes_log.write(f"{log_prefix}No note was added\n")
            elif any(keyword in current_notes.lower() for keyword in suspicious_keywords):
                odd_notes_log.write(f"{log_prefix}Suspicious notes: {row.notes}\n")